In [93]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import math, time
import time, random, pandas as pd
from selenium.common.exceptions import ElementClickInterceptedException, NoSuchElementException,StaleElementReferenceException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
from bs4 import BeautifulSoup
import time, urllib.request


In [100]:

options = Options()
options.add_argument('--window-size=782,824') 
options.add_argument('--window-position=-8,0')
options.add_experimental_option("detach", True)

search = input('검색어:')
cnt = int(input('스크래핑 할 건수는 몇건입니까?: '))
page_cnt = math.ceil(cnt / 10)  # 크롤링 할 전체 페이지 수

now = time.localtime()
date_format = '%04d%02d%02d'%(now.tm_year, now.tm_mon, now.tm_mday)
f_dir = f'{os.getcwd()}\\{search}여행기사_{cnt}건_{date_format}'
os.makedirs(f_dir)

URL = 'https://korean.visitkorea.or.kr/search/search_list.do?keyword='+search
driver = webdriver.Chrome(options=options)
driver.get(URL)
time.sleep(2)



검색어:전주
스크래핑 할 건수는 몇건입니까?: 13


In [101]:
# 여행기사 더보기 클릭
# driver.find_element(By.CSS_SELECTOR, ".more_view").click() 중복주의
driver.find_element(By.CSS_SELECTOR, "#s_recommend > .more_view > a").click()

In [102]:
title_list = []
contents_list = []
img_url_list = []

def page_work():
    result = driver.find_elements(By.CSS_SELECTOR,'#search_result .tit>a')
    global contents_no, cnt
    global title_list, contents_list, img_url_list
    
    for item in result:
        contents_no += 1
        
        if contents_no <= cnt :    
            print(f'[콘텐츠 {contents_no}]')  
            item.send_keys(Keys.ENTER) # .click()은 에러 잘남

            time.sleep(2)
            
            # 이미지 추출을 위해 미리 스크롤
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
            html = driver.page_source
            html_dom = BeautifulSoup(html, 'lxml')

            title = html_dom.find(id='topTitle')
            title_list.append(title.text)
            print(title.text)
            
            img_tag_list = html_dom.select('.img_typeBox img')
            img_url_list = [item['src'] for item in img_tag_list]

            contents = driver.find_elements(By.CLASS_NAME, 'txt_p')
            contents_merge = ' '.join([item.text for item in contents])        
            contents_list.append(contents_merge)           
            
            driver.back()
            time.sleep(2)     


In [103]:
def file_export():

    DF = pd.DataFrame({"제목":title_list, "내용":contents_list})
    filename = f'{search}여행기사_{cnt}건_{date_format}.xlsx'
    DF.to_excel(f_dir+'\\'+filename)
    print(f'====== {page_no} 페이지 {filename} 파일 저장 완료 ======')


    no = 0
    for src in img_url_list:
        # 다운로드  (주소, 파일이름)
        urllib.request.urlretrieve(src, f'{f_dir}\\{page_no}_{no}.jpg')
        no += 1
    print(f'====== {page_no} 페이지 {f_dir} 디렉토리 이미지 저장 완료 ======')

In [104]:
contents_no = 0
today = time.localtime()
print('스크래핑 프로그램 실행')

for page_no in range(1, page_cnt+1):
    print(f'====== {page_no} 페이지 스크래핑 시작 ======')
    page_work()
    print(f'====== {page_no} 페이지 스크래핑 완료 ======')

    DF = pd.DataFrame({"제목":title_list, "내용":contents_list})
    filename = f'{search}여행기사_{cnt}건_{today.tm_year}{today.tm_mon}{today.tm_mday}.xlsx'
    DF.to_excel(filename)
    print(f'====== {page_no} 페이지 {filename}파일 저장 완료 ======')

    if page_no < page_cnt:
        next = driver.find_element(By.CSS_SELECTOR, f'a[id="{page_no+1}"]')
        next.click
        
    # try    
            
    #except StaleElementReferenceException:
     #      clickUnderTen.click()
            
            
       # except ElementClickInterceptedException:
        # driver.find_elements_by_css_selector("[data-testid='loadingButton']")[0].sendkeys(Keys.ENTER)
        #    clickUnderTen.click()
        
print('스크래핑 프로그램 종료')
driver.close()

스크래핑 프로그램 실행
====== 1 페이지 스크래핑 시작 ======
[콘텐츠 1]
낭만 가득한 여행 천국, 전주
[콘텐츠 2]
전주 여행, 기차 타고 떠나기 좋은 가을 낭만 여행
[콘텐츠 3]
[살아보기 여행-전주편] ‘반반(半半) 전주’ 
[콘텐츠 4]
비빔밥 와플부터 콩나물 아이스크림까지! 요즘 뜨는 전주의 맛 6
[콘텐츠 5]
풋풋한 청춘 서사 〈스물다섯 스물하나〉 속 전주 서학동예술마을과 한벽굴
[콘텐츠 6]
[전주] 천천히, 오래 지켜가는 지속 가능한 전주, 사랑가득 한옥 펜션
[콘텐츠 7]
[전주] 슬로시티 전주에서 바삐 달려왔던 일상에 쉼표를, 전주 정가한옥
[콘텐츠 8]
가장 한국전인 멋의 고장, 전주한옥마을
[콘텐츠 9]
전주여행, 색다르게 즐기는 전주 핫플레이스
[콘텐츠 10]
 [완(주)전(주)익(산)사이팅한 역사 길] 과거와 현대가 공존하는 곳, 전주 전주천길
====== 1 페이지 스크래핑 완료 ======
====== 1 페이지 전주여행기사_13건_2024320.xlsx파일 저장 완료 ======
====== 2 페이지 스크래핑 시작 ======
[콘텐츠 11]
낭만 가득한 여행 천국, 전주
[콘텐츠 12]
전주 여행, 기차 타고 떠나기 좋은 가을 낭만 여행
[콘텐츠 13]
[살아보기 여행-전주편] ‘반반(半半) 전주’ 
====== 2 페이지 스크래핑 완료 ======
====== 2 페이지 전주여행기사_13건_2024320.xlsx파일 저장 완료 ======
스크래핑 프로그램 종료
